<a href="https://colab.research.google.com/github/kruspe2009/volunteer_projects/blob/main/DonorSearch/DonorSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание проекта


Сервис DonorSearch поставил перед собой 2 цели: 

1.   Увеличить количество доноров
2.   Больше вовлекать существующих доноров

В качестве исходных данных предоставлены 3 таблицы:



*   Донации крови
*   Планируемые донации крови
*   Анонимная таблица доноров




# Импорт библиотек и чтение данных

In [1]:
import pandas as pd

In [2]:
donations_anon = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DonorSearch/donations_anon.csv')
plan_anon = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DonorSearch/plan_anon.csv')
users_anon = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DonorSearch/users_anon.csv')

# Знакомство с данными



## Предобработка данных

## Таблица `donations_anon`

In [3]:
#изучим данные в таблице
donations_anon.head()

,ID,ID пользователя,Класс крови,Дата донации,Дата добавления донации,Тип донации,Регион,Место стадчи,Статус донации,Есть справка
0,97920,151894,Плазма,02.06.2021,02.06.2021,Безвозмездно,"Россия, Москва",647,На модерации,Да
1,97919,156916,Тромбоциты,02.06.2021,02.06.2021,Безвозмездно,"Россия, Москва",633,На модерации,Да
2,97918,149413,Цельная кровь,01.06.2021,02.06.2021,Безвозмездно,"Россия, Приморский край, Владивосток",325,Принята,Да
3,97917,155027,Цельная кровь,02.06.2021,02.06.2021,Безвозмездно,"Россия, Саратовская область, Саратов",500,Принята,Да
4,97916,151094,Цельная кровь,23.04.2021,02.06.2021,Безвозмездно,"Россия, Свердловская область, Екатеринбург",509,Принята,Да


In [4]:
#посмотрим на столбцы
donations_anon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97874 entries, 0 to 97873
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID                       97874 non-null  int64 
 1   ID пользователя          97874 non-null  int64 
 2   Класс крови              97874 non-null  object
 3   Дата донации             97874 non-null  object
 4   Дата добавления донации  97874 non-null  object
 5   Тип донации              97874 non-null  object
 6   Регион                   97874 non-null  object
 7   Место стадчи             97874 non-null  object
 8   Статус донации           97874 non-null  object
 9   Есть справка             97874 non-null  object
dtypes: int64(2), object(8)
memory usage: 7.5+ MB


В процессе чтения таблицы с данными о донациях, возникла проблема с парсингом даты - выяснилось, что некоторые даты указаны в формате, отличном от стандартного для данного датасета. Посмотрим на них более подробно. 

In [5]:
#"стандарт" для данного датасета - 10 символов в дате
dates = set()

#отберем даты, отличные от стандарта
for date in donations_anon['Дата донации']:
  if len(date) != 10:
    dates.add(date)
    print(date)

21.04.208
27.04.201
15.12.214
24.05.207


In [6]:
#уберем некорректные данные из датасета
donations_anon = donations_anon[~donations_anon['Дата донации'].isin(dates)]

In [7]:
#по столбцу "Дата добавления донации" проблем нет
for date in donations_anon['Дата добавления донации']:
  if len(date) != 10:
    dates.add(date)
    print(date)

При попытке преобразовать дату донации из строкового формата в формат даты возвращается ошибка. По всей вероятности, помимо даты, в которой меньше символов, чем в стандартной дате, в некоторых строках указаны некорректные даты.

In [15]:
date_list = set()

for date in donations_anon['Дата донации']:
  date_split = date.split('.')
  for i in date_split:
    if int(date_split[2]) > 2021 or int(date_split[2]) < 1980:
      date_list.add(date)

In [16]:
#снова уберем некорректные данные из датасета
donations_anon = donations_anon[~donations_anon['Дата донации'].isin(date_list)]

In [19]:
#преобразование дат из строкового типа в тип даты
donations_anon['Дата донации'] = pd.to_datetime(donations_anon['Дата донации'])
donations_anon['Дата добавления донации'] = pd.to_datetime(donations_anon['Дата добавления донации'])

In [24]:
#посмотрим на дипазон дат добавления донации, так как с датой самой донации ранее мы уже обнаружили проблемы
print('Минимальная дата добавления донации:', donations_anon['Дата добавления донации'].min())
print('Максимальная дата добавления донации:',  donations_anon['Дата добавления донации'].max())

Минимальная дата добавления донации: 2020-01-12 00:00:00
Максимальная дата добавления донации: 2021-12-05 00:00:00
